# Gaussian mixtures estimation using VB & EM algorithms

This notebook aims at comparing Variational Bayes and Expectation Maximization algorithms for estimating Gaussian mixtures. It contains a numerical illustration of Algorithm 3 of the research paper 'Consistency of Variational Bayes Inference for Estimation and Model Selection in Mixtures' (blablabla).

We estimate a simple unit-variance Gaussian mixture using the Coordinate Descent Variational Inference algorithm for $\alpha=0.5$. We compare it to the case $\alpha=1$ (equivalent to the classical CAVI algorithm) and to EM algorithm.

We compare our algorithms using the Mean Average Error (MAE) between the estimates and the true parameters. We consider 10 different unit-variance Gaussian mixtures which parameters are generated independently from a Dirichlet distribution and Gaussians (see the code below). From these mixtures, we create 10 different datasets which contain 1000 i.i.d. realizations of the corresponding mixtures. For each dataset, we run each algorithm 5 times and keep the one with the lowest MAE in order to avoid situations where the initialization leads to a local optimum. Then, we average the resulting MAEs over the different datasets to obtain the final values of the MAE. We also record the standard deviation of the MAE over the different datasets.

### Import packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import *
import time
% matplotlib inline

### Coordinate Descent Variational Inference algorithm

In [2]:
# Coordinate Descent Variational Inference CDVI algorithm

def CDVI(K, prior_var, prior_alpha, n, data, alpha):
    
    # Random initialization
    mu_mean = np.random.normal(0, prior_var,K)
    mu_var = abs(np.random.normal(0, prior_var,K))
    phi = abs(np.random.normal(0, prior_var,K))      
    weights = np.zeros((n, K))
    col = np.random.choice(K,n)
    for j in range(K):
        weights[col==j,j]=1
    
    
    # Number of iterations counter
    ii=0
    
    # Initiate CDVI iterations
    while(True):
        mu_mean_prev = mu_mean.copy()
        phi_prev = phi.copy()
        
        # mixture model parameter update step
        for j in range(K):
            sn = np.sum(weights[:,j]*data)
            sd = np.sum(weights[:,j])
            mu_mean[j] = alpha*sn/((1/prior_var**2) + alpha*sd)
            mu_var[j] = 1/((1/prior_var**2) + alpha*sd)

        # categorical vector update step
        weights_int = np.zeros((n,K))
        for j in range(K):
            weights_int[:,j] = np.exp(psi(phi[j]) - psi(sum(phi)) + mu_mean[j]*data[:] - (mu_var[j] + mu_mean[j]**2)/2)
        for k in range(K):
            weights[:,k] = weights_int[:,k]/weights_int.sum(axis=1)
                
        # weight parameter update step
        for j in range(K):
            phi[j] = prior_alpha[j] + alpha*np.sum(weights[:,j])
                    
        ii+=1
                
        # convergence of variational estimators
        if (np.dot(np.array(mu_mean_prev) - np.array(mu_mean), np.array(mu_mean_prev) - np.array(mu_mean)) < 1e-10):
            if (np.dot(np.array(phi_prev) - np.array(phi), np.array(phi_prev) - np.array(phi)) < 1e-10):
                break
            
    #print('Algorithm converges in', ii, 'iterations')
    
    mu_mean = np.array(mu_mean)
    mu_var = np.array(mu_var)
    phi = np.array(phi)

    # return final estimates
    return(mu_mean, mu_var, weights, phi)

### Hyperparameters definition

In [3]:
# Priors hyperparameters
prior_var = 10
prior_alpha = [2/3,2/3,2/3]

# Dimensions of the problem
K = 3
n = 1000

### Comparison with EM

In [4]:
# Compute running time
start = time.time()


# Define Mean Average Errors & Standard Dev of the weight and parameter components estimators for the three algorithms

serie_poids_aVB=[]
serie_param_aVB=[[] for _ in range(K)]

serie_poids_CAVI=[]
serie_param_CAVI=[[] for _ in range(K)]

serie_poids_EM=[]
serie_param_EM=[[] for _ in range(K)]

# 'serie_poids_algorithm' will be a list containing the MAE of the weight for each of the 10 created datasets
# 'serie_param_algorithm' will be a list containing the list of components MAEs for each of the 10 created datasets


# For 10 different datasets
for i in range(10):
    
    # Draw parameters randomly
    p_true = np.random.dirichlet(prior_alpha)
    mu = np.random.normal(0, prior_var,K)
    # Simulate data : pick component and sample from the corresponding distribution
    N = np.random.choice(K, n, p=p_true)
    data = np.random.normal(mu[N], 1, n)
    
    # Prediction errors for the CDVI algorithm for this dataset (best of 5 random initializations to avoid local extrema)
    # Create a temporary list that will contain the 5 values of MAEs for the 5 random intializations
    serie_poids_temp=[]
    serie_param_temp=[[] for _ in range(K)]
    for l in range(5):
        # Define estimated parameters for 5 different initializations
        mu_mean, mu_var, weights, phi = CDVI(K, prior_var, prior_alpha, n, data, 0.5)
        poids = list(phi/sum(phi))
        # Add MAE obtained for this initialization to the temporay list
        serie_poids_temp.append(sum(abs(np.array(sorted(poids))-np.array(sorted(p_true)))))
        for j in range(K):
            serie_param_temp[j].append(abs(sorted(mu_mean)[j]-sorted(mu)[j]))
    
    # Best of 5 initializations
    serie_poids_aVB.append(min([serie_poids_temp[l] for l in range(5)]))
    for j in range(K):
        serie_param_aVB[j].append(min([serie_param_temp[j][l] for l in range(5)]))
        
    # Prediction errors for the CAVI algorithm for this dataset (best of 5 random initializations to avoid local extrema)
    # Create a temporary list that will contain the 5 values of MAEs for the 5 random intializations
    serie_poids_temp=[]
    serie_param_temp=[[] for _ in range(K)]
    for l in range(5):
        # Define estimated parameters for 5 different initializations
        mu_mean, mu_var, weights, phi = CDVI(K, prior_var, prior_alpha, n, data, 1)
        poids = list(phi/sum(phi))
        serie_poids_temp.append(sum(abs(np.array(sorted(poids))-np.array(sorted(p_true)))))
        # Add MAE obtained for this initialization to the temporay list
        for j in range(K):
            serie_param_temp[j].append(abs(sorted(mu_mean)[j]-sorted(mu)[j]))
            
    # Best of 5 initializations
    serie_poids_CAVI.append(min([serie_poids_temp[l] for l in range(5)]))
    for j in range(K):
        serie_param_CAVI[j].append(min([serie_param_temp[j][l] for l in range(5)]))
    
    # Prediction errors for the EM algorithm for this dataset (best of 5 random initializations to avoid local extrema)
    # Create a temporary list that will contain the 5 values of MAEs for the 5 random intializations
    serie_poids_temp=[]
    serie_param_temp=[[] for _ in range(K)]
    for l in range(5):
        # Define estimated parameters for 5 different initializations
        mu_mean, mu_var, weights, phi = CDVI(K, prior_var, prior_alpha, n, data, 0.5)
        poids = list(phi/sum(phi))
        serie_poids_temp.append(sum(abs(np.array(sorted(poids))-np.array(sorted(p_true)))))
        # Add MAE obtained for this initialization to the temporay list
        for j in range(K):
            serie_param_temp[j].append(abs(sorted(mu_mean)[j]-sorted(mu)[j]))
            
    # Best of 5 initializations
    serie_poids_EM.append(min([serie_poids_temp[l] for l in range(5)]))
    for j in range(K):
        serie_param_EM[j].append(min([serie_param_temp[j][l] for l in range(5)]))


# Average the MAEs and the standard devs accross the different datasets and obtain the final values
MAE_poids_aVB=np.mean(serie_poids_aVB)
std_poids_aVB=np.std(serie_poids_aVB)
MAE_poids_CAVI=np.mean(serie_poids_CAVI)
std_poids_CAVI=np.std(serie_poids_CAVI)
MAE_poids_EM=np.mean(serie_poids_EM)
std_poids_EM=np.std(serie_poids_EM)
MAE_param_aVB=[np.mean(serie_param_aVB[k]) for k in range(K)]
std_param_aVB=[np.std(serie_param_aVB[k]) for k in range(K)]
MAE_param_CAVI=[np.mean(serie_param_CAVI[k]) for k in range(K)]
std_param_CAVI=[np.std(serie_param_CAVI[k]) for k in range(K)]
MAE_param_EM=[np.mean(serie_param_EM[k]) for k in range(K)]
std_param_EM=[np.std(serie_param_EM[k]) for k in range(K)]

# Running time
end = time.time()
print('Running time :', end - start, 'seconds')

Running time : 8.196572542190552 seconds


### Print the final results

In [5]:
print('alpha-VB : MAE :', 'weight :', MAE_poids_aVB, ', components :', MAE_param_aVB)
print('         : std :', 'weight :', std_poids_aVB, ', components :', std_param_aVB)
print()
print('CAVI     : MAE :', 'weight :', MAE_poids_CAVI, ', components :', MAE_param_CAVI)
print('         : std :', 'weight :', std_poids_CAVI, ', components :', std_param_CAVI)
print()
print('EM       : MAE :', 'weight :', MAE_poids_EM, ', components :', MAE_param_EM)
print('         : std :', 'weight :', std_poids_EM, ', components :', std_param_EM)

alpha-VB : MAE : weight : 0.0581608631127 , components : [0.78785634773818169, 1.0542170095085945, 0.53166271677534505]
         : std : weight : 0.036179092105 , components : [2.1396219815095576, 1.8089346878153001, 1.1308546560072157]

CAVI     : MAE : weight : 0.0520629745943 , components : [0.787908956687568, 0.68138240823362584, 0.15583486210411968]
         : std : weight : 0.0376684661035 , components : [2.1396129253608756, 1.5792407336364849, 0.13201899595516536]

EM       : MAE : weight : 0.0581608133597 , components : [0.78779631157391716, 1.0542169568340523, 0.53166265848076444]
         : std : weight : 0.0361790485619 , components : [2.1396421840829718, 1.8089347150782544, 1.1308546761502078]
